In [ ]:
import numpy as np
import pandas as pd

import os   
import open3d as o3d
import re
import matplotlib.pyplot as plt

In [2]:
biomass = np.loadtxt("../data/biomass/biomass.txt", dtype=str)
biomass = np.array([string.replace(",", ".") for string in biomass])
biomass = biomass.astype(float)
path = "../data/biomass/biomass.npy"
np.save(path, biomass)

In [ ]:
plt.hist(np.log(biomass), bins=10)
plt.savefig(fname="plot/biomass_hist",format="svg", bbox_inches="tight")

In [4]:
biomass = np.log(biomass)
mean_logmass = biomass.mean()
std_logmass = biomass.std()
biomass = (biomass - biomass.mean()) / biomass.std()

In [ ]:
biomass.std()

In [6]:
dir = "../data/biomass/trees"
samples_list = os.listdir(dir)
sample_numbers = [re.findall(r'\d+', sample)  for sample in samples_list]
sample_numbers = [int(sample[0]) for sample in sample_numbers]
sort_idx = np.array(sample_numbers).argsort()
samples_list = [os.path.join(dir, sample) for sample in samples_list]
samples_list = np.array(samples_list)[sort_idx]

In [7]:
VAL=1000
def voxelize(points, voxelsize=0.10):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    #print(f"Voxelize the point cloud with a voxel size of {voxelsize}")
    min_bound, max_bound = np.array([-VAL, -VAL, -VAL]), np.array([VAL, VAL, VAL])
    downpcd, _, idx = pcd.voxel_down_sample_and_trace(voxelsize, min_bound, max_bound)
    print(f"Previous size: {points.shape[0]}, new size {np.asarray(downpcd.points).shape[0]}")
    return np.array(downpcd.points)

In [ ]:
savedir = "../data/trees/biomass-trees"

for idx, (mass, sample) in enumerate(zip(biomass, samples_list)):
    path = os.path.join(savedir, str(idx) + ".npy")
    #if os.path.exists(path):
    #    continue
    pt = np.loadtxt(sample)
    pt = voxelize(pt)
    
    np.save(path, (mass, pt))
    #print(idx, "done")

# explore simple methods for prediction

In [ ]:
import torch
loss = torch.nn.MSELoss()
prediction = biomass.mean()
loss(torch.tensor(prediction), torch.tensor(biomass))


In [ ]:
%cd ..


In [ ]:
import utils.parser as parser
from utils.config import *
import tools.builder as builder

argv = ["--config", "cfgs/regression/biomass_treeset.yaml"]
args = parser.get_args(argv)
config = get_config(args)
args.distributed = False
print(config.dataset.train)
config.dataset.train.others.bs = 1
config.dataset.test.others.bs = 1
config.dataset.test.others.model = config.model
config.dataset.train.others.model = config.model

In [ ]:
_, loader = builder.dataset_builder(args, config.dataset.train)
trainset = loader.dataset
_, loader = builder.dataset_builder(args, config.dataset.test)
testset = loader.dataset
testset.normalization=False

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression
from torchvision import transforms

In [11]:
trainset.transforms = transforms.Compose([])
trainset.token_transforms = transforms.Compose([])
trainset.normalization = True
testset.normalization = True

In [ ]:
feat = np.empty((len(trainset), 3))
y = np.empty(len(trainset))
print(y.shape)
for i, (neighborhood, center, target) in enumerate(trainset):
    points, target = center.numpy(), target.numpy()
    ptcloud = neighborhood.reshape(-1, 3).T
    x, ys, z = ptcloud.numpy()
    feat[i, 0:3]  = np.ptp(x), np.ptp(ys) , np.ptp(z)
    y[i] = target

In [ ]:
reg = LinearRegression().fit(feat, y)
reg.score(feat, y)

In [14]:
feattest = np.empty((len(testset), 3))
ytest = np.empty(len(testset))
for i, (neighborhood, center, target) in enumerate(testset):
    points, target = center.numpy(), target.numpy()
    ptcloud = neighborhood.reshape(-1, 3).T
    x, ys, z = ptcloud.numpy()
    feattest[i, 0:3] = np.ptp(x), np.ptp(ys) , np.ptp(z)
    ytest[i] = target


In [ ]:
prediction = reg.predict(feattest)
np.mean((prediction -ytest)**2)

# Scale up predictions to original scale

In [ ]:
ytest

In [ ]:
predictions = np.load("experiments/regression/biomass_treeset/onlycenterfps/bestresultsforregression/predictions.npy").reshape(-1)
print(loss(torch.tensor(predictions), torch.tensor(ytest)))

res = (predictions - ytest)**2
res

In [ ]:
np.sum(res) /len(res)

In [ ]:
plt.scatter(ytest, predictions)
plt.xlabel("normalized log tree volume")
plt.xlim(-2, 1.7)
plt.ylim(-2, 1.7)
plt.ylabel("predicted  normalized log tree volume")
x1, y1 = [-2, 1.7], [-2, 1.7]
plt.plot(x1, y1, "--")
plt.savefig(fname="custom/plot/log_predvsactual.svg",format="svg", bbox_inches="tight")

In [52]:
import numpy as np
import pandas as pd
import os   
import open3d as o3d
import re

In [53]:
biomass = np.load("../data/biomass/biomass.npy")
biomass = np.log(biomass)
mean = biomass.mean()
std = biomass.std()
#biomass = (biomass - biomass.mean()) / biomass.std()

In [54]:
pred = np.exp(predictions * std_logmass + mean_logmass)
#rescale predictions and values to original scale
target = np.exp(ytest * std_logmass + mean_logmass)

In [ ]:
plt.scatter((target), (pred))
plt.xlabel("tree volume")
plt.ylabel("predicted tree volume")
plt.xlim(-1, 20)
plt.ylim(-1, 20)
x1, y1 = [-1, 20], [-1, 20]
plt.plot(x1, y1, "--")
plt.savefig(fname="custom/plot/predvsactual.svg",format="svg", bbox_inches="tight")

In [ ]:
loss(torch.tensor(pred), torch.tensor(target))

In [ ]:
np.median(res/ target)

In [ ]:
np.mean(res / target)